# NanoGPT

Training a decoder-only model (NanoGPT) to genereate text in Shakespear stype

## Install Modules

In [1]:
# Tiktoken for tokenization
!pip install tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.2 MB/s eta 0:00:00


## Import Modules

In [2]:
# Standard Library Imports
import os
import math
import time
import inspect
from dataclasses import dataclass

# Third-Party Imports
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F

## Transformer Achitecture

In [3]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANGPT_SCALE_INIT = 1
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        # nh is "number of heads", hs is "head size", and C (number of channels) = nh * hs
        # e.g. in GPT-2 (124M), n_head=12, hs=64, so nh*hs=C=768 channels in the Transformer
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing
        self.transformer.wte.weight = self.lm_head.weight

        # weight initialization
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean = 0.0, std = std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

## Configuration Parameters

In [4]:
@dataclass
class GPTConfig:
    block_size: int = 1024  # max sequence length
    vocab_size: int = 50257 # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
    n_layer: int = 12       # number of layers
    n_head: int = 12        # number of heads
    n_embd: int = 768       # embedding dimension

## DataLoader

In [5]:
class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # at init load tokens from disk and store them in memory
        with open('input.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded {len(self.tokens)} tokens')
        print(f'1 epoch = {len(self.tokens) // (B * T)} batches')

        # state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + B * T + 1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B*T
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

## Device Configutration

In [6]:
# Add CUDA check and memory info
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Modify the device selection
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

CUDA available: True
CUDA device: Tesla T4
Total GPU memory: 15.84 GB
Using device: cuda


## Model Training

In [10]:
# SEED
torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

# Model
model = GPT(GPTConfig())
model.to(device)

# Print total model parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params:,}\n")

# Dataloader
train_loader = DataLoaderLite(B = 4, T = 1024)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4)

step = 1
while True:
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()
    optimizer.step()

    if device == 'cuda' and step % 100 == 0:  # Print memory every 100 steps
        print(f"GPU Memory: {torch.cuda.memory_allocated() / 1e9:.2f}GB / {torch.cuda.memory_reserved() / 1e9:.2f}GB")

    if step % 100 == 0:  # Print details every 100 steps
        print(f'step {step:,} | loss: {loss.item():.4f}')
        print("    ")

    if loss.item() < 0.09:
        print(f'\nReached target loss! Final loss: {loss.item():.4f} at step {step:,}')
        save_path = 'gpt_model.pt'
        torch.save(model.state_dict(), save_path)
        print(f"Model saved to {save_path}")
        break

    step += 1

Total model parameters: 124,439,808

loaded 338025 tokens
1 epoch = 82 batches
GPU Memory: 3.75GB / 12.74GB
step 100 | loss: 5.6404
    
GPU Memory: 3.75GB / 12.74GB
step 200 | loss: 5.5134
    
GPU Memory: 3.75GB / 12.74GB
step 300 | loss: 5.3910
    
GPU Memory: 3.75GB / 12.74GB
step 400 | loss: 5.2524
    
GPU Memory: 3.75GB / 12.74GB
step 500 | loss: 4.6032
    
GPU Memory: 3.75GB / 12.74GB
step 600 | loss: 4.9332
    
GPU Memory: 3.75GB / 12.74GB
step 700 | loss: 4.6553
    
GPU Memory: 3.75GB / 12.74GB
step 800 | loss: 4.7186
    
GPU Memory: 2.92GB / 12.74GB
step 900 | loss: 4.1250
    
GPU Memory: 3.75GB / 12.74GB
step 1,000 | loss: 4.4559
    
GPU Memory: 3.75GB / 12.74GB
step 1,100 | loss: 4.5999
    
GPU Memory: 2.92GB / 12.74GB
step 1,200 | loss: 4.1833
    
GPU Memory: 3.75GB / 12.74GB
step 1,300 | loss: 3.9645
    
GPU Memory: 3.75GB / 12.74GB
step 1,400 | loss: 3.8615
    
GPU Memory: 2.92GB / 12.74GB
step 1,500 | loss: 4.2542
    
GPU Memory: 3.75GB / 12.74GB
step 1,600

## Save Model

In [11]:
# Print total model parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params:,}")

# Save the model
save_path = 'nano_gpt_model.pt'
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Total model parameters: 124,439,808
Model saved to nano_gpt_model.pt


## Inference

In [12]:
# STOP
num_return_sequences = 5
max_length = 30

# Text generation
print("\nGenerating text samples...")
enc = tiktoken.get_encoding('gpt2')
context = "Once upon a time"
x = torch.tensor([enc.encode(context)], dtype=torch.long, device=device)

max_length = 100  # Generate 100 tokens
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

while x.size(1) < max_length:
    with torch.no_grad():
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits = model(x)[0]
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, num_samples=1)
        xcol = torch.gather(topk_indices, -1, ix)
        x = torch.cat([x, xcol], dim=1)

print("\nGenerated text:")
tokens = x[0].tolist()  # Take first sequence
decoded = enc.decode(tokens)
print(f"\n{decoded}")


Generating text samples...

Generated text:

Once upon a time Bol sorrow did man sp;
First patience the court

SLYry; I may stay that been resARERSAS Mician:
Vols of Somerset, hoiestPut by his I besiege!
Then, as I am as yet he hath been in Padua but the world:

To look from him.
With joyful for these sport in some cause:
Of this same, by your honour and not, and daughter, young Warwick in
